In [1]:
import os
import requests
import re
import csv
import json
import time
import html
import string
import itertools
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from geojson import Feature, FeatureCollection, Point

In [14]:
URL = 'https://escapefromtarkov.fandom.com/wiki/Quests'
data = requests.get(URL, stream=True)
quests = []

if (data.status_code == 200):
    for line in data.iter_lines():
        match = re.findall('mw-selflink selflink">Quests<', str(line))
        if match:
            match2 = re.findall('<a href="/wiki/([^"]*)" title="([^"]*)">', str(line))
            if match2:
                quests = match2[1:-1]

In [15]:
quests

[('Debut', 'Debut'),
 ('Checking', 'Checking'),
 ('Shootout_Picnic', 'Shootout Picnic'),
 ('Delivery_from_the_Past', 'Delivery from the Past'),
 ('BP_Depot', 'BP Depot'),
 ('Bad_Rep_Evidence', 'Bad Rep Evidence'),
 ('Ice_Cream_Cones', 'Ice Cream Cones'),
 ('Postman_Pat_-_Part_1', 'Postman Pat - Part 1'),
 ('Shaking_up_the_Teller', 'Shaking up the Teller'),
 ('The_Punisher_-_Part_1', 'The Punisher - Part 1'),
 ('The_Punisher_-_Part_2', 'The Punisher - Part 2'),
 ('The_Punisher_-_Part_3', 'The Punisher - Part 3'),
 ('The_Punisher_-_Part_4', 'The Punisher - Part 4'),
 ('The_Punisher_-_Part_5', 'The Punisher - Part 5'),
 ('The_Punisher_-_Part_6', 'The Punisher - Part 6'),
 ('Polikhim_Hobo', 'Polikhim Hobo'),
 ('Big_Customer', 'Big Customer'),
 ('No_Offence', 'No Offence'),
 ('Grenadier', 'Grenadier'),
 ('Perfect_Mediator', 'Perfect Mediator'),
 ('Insomnia', 'Insomnia'),
 ('Test_Drive_-_Part_1', 'Test Drive - Part 1'),
 ('Regulated_Materials', 'Regulated Materials'),
 ('The_Bunker_-_Part_1'

In [74]:
questData = {}
for quest in quests:
    questData[quest[1]] = {
        'url': quest[0]
    }

idx = 0
for questTuple in quests:
    idx += 1
    if idx % 10 == 0:
        print(idx, '/', len(quests))
    
    quest = questTuple[1]
    URL = 'https://escapefromtarkov.fandom.com/wiki/' + questTuple[0]
    data = requests.get(URL, stream=True)
    if (data.status_code == 200):
        questData[quest]['id'] = idx
        questData[quest]['reqs'] = {
            'level': -1,
            'traders': [],
            'quests': []
        }
        questData[quest]['maps'] = []
        questData[quest]['objectives'] = []
        objectiveLine = False

        for line in data.iter_lines():
            matchCustoms = re.findall('on Customs | on <a href="/wiki/Customs"', str(line))
            if matchCustoms and 'customs' not in questData[quest]['maps']:
                questData[quest]['maps'].append('customs')

            matchLab = re.findall('on Lab | on <a href="/wiki/Shoreline"', str(line))
            if matchLab and 'lab' not in questData[quest]['maps']:
                questData[quest]['maps'].append('lab')

            matchInterchange = re.findall('on Interchange | on <a href="/wiki/Interchange"', str(line))
            if matchInterchange and 'interchange' not in questData[quest]['maps']:
                questData[quest]['maps'].append('interchange')

            matchWoods = re.findall('on Woods | on <a href="/wiki/Woods"', str(line))
            if matchWoods and 'woods' not in questData[quest]['maps']:
                questData[quest]['maps'].append('woods')

            matchReserve = re.findall('on Reserve | on <a href="/wiki/Reserve"', str(line))
            if matchReserve and 'reserve' not in questData[quest]['maps']:
                questData[quest]['maps'].append('reserve')

            matchShoreline = re.findall('on Shoreline | on <a href="/wiki/Shoreline"', str(line))
            if matchShoreline and 'shoreline' not in questData[quest]['maps']:
                questData[quest]['maps'].append('shoreline')

            matchLighthouse = re.findall('on Lighthouse | on <a href="/wiki/Lighthouse"', str(line))
            if matchLighthouse and 'lighthouse' not in questData[quest]['maps']:
                questData[quest]['maps'].append('lighthouse')

            matchFactory = re.findall('on Factory | on <a href="/wiki/Factory"', str(line))
            if matchFactory and 'factory' not in questData[quest]['maps']:
                questData[quest]['maps'].append('factory')
                
                
            matchReqLvl = re.findall('Must be level (\d*)', str(line))
            if matchReqLvl:
                questData[quest]['reqs']['level'] = matchReqLvl[0]
                
            matchReqTrader = re.findall('Obtain level (\d*) loyalty with (\S*)', str(line))
            if matchReqTrader and len(matchReqTrader[0]) > 1 and len(matchReqTrader[0][1]) > 2 and (matchReqTrader[0][0], matchReqTrader[0][1]) not in questData[quest]['reqs']['traders']:
                questData[quest]['reqs']['traders'].append((matchReqTrader[0][0], matchReqTrader[0][1]))

            matchReqQuests = re.findall('Previous:<br /><a href="/wiki/[^"]*" title="([^"]*)"', str(line))
            if matchReqQuests:
                questData[quest]['reqs']['quests'].append(matchReqQuests[0])
            
                matchReqQuests2 = re.findall(matchReqQuests[0] + '</a><br />or<br /><a href="/wiki/[^"]*" title="([^"]*)"', str(line))
                if matchReqQuests2:
                    questData[quest]['reqs']['quests'].append(matchReqQuests2[0])

            
            matchObjectives = re.findall('id="Objectives"', str(line))
            if matchObjectives:
                objectiveLine = True

            if objectiveLine and not matchObjectives:
                matchEndObjectives = re.findall('<h2>', str(line))
                if not matchEndObjectives:
                    matchObjective = re.findall('(?:>)(?:\s*|([^><]+))(?:<)', str(line)[2:])
                    questData[quest]['objectives'].append(''.join(matchObjective))
                else:
                    objectiveLine = False


questData

10 / 258
20 / 258
30 / 258
40 / 258
50 / 258
60 / 258
70 / 258
80 / 258
90 / 258
100 / 258
110 / 258
120 / 258
130 / 258
140 / 258
150 / 258
160 / 258
170 / 258
180 / 258
190 / 258
200 / 258
210 / 258
220 / 258
230 / 258
240 / 258
250 / 258


{'Debut': {'url': 'Debut',
  'id': 1,
  'reqs': {'level': -1, 'traders': [], 'quests': []},
  'maps': ['customs'],
  'objectives': ['Eliminate 5 Scavs on Customs',
   'Obtain and hand over 2 MP-133 12ga shotguns']},
 'Checking': {'url': 'Checking',
  'id': 2,
  'reqs': {'level': '2', 'traders': [], 'quests': ['Debut']},
  'maps': ['customs'],
  'objectives': ['Obtain the Bronze pocket watch on a chain on Customs',
   '(Optional) Obtain the key to the fuel tanker truck',
   'Hand over the pocket watch']},
 'Shootout Picnic': {'url': 'Shootout_Picnic',
  'id': 3,
  'reqs': {'level': '3', 'traders': [], 'quests': ['Checking']},
  'maps': ['woods'],
  'objectives': ['Eliminate 15 Scavs on Woods']},
 'Delivery from the Past': {'url': 'Delivery_from_the_Past',
  'id': 4,
  'reqs': {'level': '5', 'traders': [], 'quests': ['Checking']},
  'maps': ['customs'],
  'objectives': ["Obtain the secure folder in the Tarcone Director\\'s office at the Customs terminal warehouse",
   'Stash the package 

In [55]:
questData

{'Debut': {'url': 'Debut'},
 'Checking': {'url': 'Checking',
  'reqlvl': '2',
  'maps': ['customs', 'customs', 'customs', 'customs'],
  'objectives': ['Obtain the Bronze pocket watch on a chain on Customs',
   '(Optional) Obtain the key to the fuel tanker truck',
   'Hand over the pocket watch']},
 'Shootout Picnic': {'url': 'Shootout_Picnic',
  'reqlvl': '3',
  'maps': ['woods', 'woods', 'woods', 'woods'],
  'objectives': ['Eliminate 15 Scavs on Woods']},
 'Delivery from the Past': {'url': 'Delivery_from_the_Past',
  'reqlvl': '5',
  'maps': ['customs'],
  'objectives': ["Obtain the secure folder in the Tarcone Director\\'s office at the Customs terminal warehouse",
   'Stash the package in the Factory break room (2nd floor near Gate 3)',
   'Survive and extract from Factory']},
 'BP Depot': {'url': 'BP_Depot',
  'reqlvl': '5',
  'maps': ['customs',
   'customs',
   'customs',
   'customs',
   'customs',
   'customs',
   'customs'],
  'objectives': ['Locate and mark the first fuel tan

In [75]:
with open('quest_data.json', 'w') as f:
    json.dump(questData, f)

In [76]:
for quest in questData.keys():
    if not len(questData[quest]['maps']):
        print(questData[quest])

{'url': 'The_Punisher_-_Part_5', 'id': 14, 'reqs': {'level': '20', 'traders': [], 'quests': ['The Punisher - Part 4']}, 'maps': [], 'objectives': ['Find a AK-74N assault riflein raid', 'Hand over the AK-74N', 'Find a Colt M4A1 assault riflein raid', 'Hand over the M4A1', 'Find 2 Makarov PM pistolsin raid', 'Hand over the 2 pistols', 'Eliminate 10 PMC operatives while wearing a PACA body armor and 6B47 helmet']}
{'url': 'The_Punisher_-_Part_6', 'id': 15, 'reqs': {'level': '21', 'traders': [], 'quests': ['The Punisher - Part 5']}, 'maps': [], 'objectives': ['', '', '', 'Despite the objective wording, the dogtags do not need to be found in raid!', '', 'Eliminate 15 PMC operatives while using SVD rifle (excluding Factory location)', 'Find 7 BEAR PMC dogtagsin raid', 'Hand over the 7 BEARdogtags', 'Find 7 USEC PMC dogtagsin raid', 'Hand over the 7 USECdogtags']}
{'url': 'No_Offence', 'id': 18, 'reqs': {'level': -1, 'traders': [], 'quests': ['Chemical - Part 4', 'Out of Curiosity']}, 'maps':